<a href="https://colab.research.google.com/github/rim-yu/SGU_ANT6320_Assignment/blob/master/Exercise05_tabular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

lesson4 : https://course.fast.ai/videos/?lesson=4

detailed note : https://github.com/hiromis/notes/blob/master/Lesson4.md

very detailed note i found : https://medium.com/analytics-vidhya/neural-networks-for-tabular-data-touching-the-untouched-efd66c950463

# Tabular models

"Tabular data" is just data that has been formatted as a table, with rows and columns (like a spreadsheet). You can easily read a tabular data file into pandas, even directly from a URL.


In [0]:
from fastai.tabular import *

Tabular data should be in a Pandas `DataFrame`.

In [2]:
path = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(path/'adult.csv')

In [17]:
path

PosixPath('/root/.fastai/data/adult_sample')

**categorical vs. continuous**

For categorical variables, we're going to be using something called embeddings which we'll be learning about later today. 

For continuous variables, they could just be sent into the neural net just like pixels in a neural net can. 

In [0]:
dep_var = 'salary'
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'] 
# categorical names.

cont_names = ['age', 'fnlwgt', 'education-num'] 
# continuous names.
# 'fnlwgt' means final weight which is the number of units in the target population that the responding unit represents.

procs = [FillMissing, Categorify, Normalize]
# FillMissing : Look for missing values and deal with them some way.
# Categorify : Find categorical variables and turn them into Pandas categories.
# Normalize : Do a normalization ahead of time which is to take continuous variables and 
# subtract their mean and divide by their standard deviation so they are zero-one variavbles. 


**Transforms vs. processes**

Transforms in computer vision do things like flip a photo on its axis, turn it a bit, brighten it, or normalize it. 

But for **tabular data**, instead of having transforms, we have things called processes. And they're nearly identical but the key difference, which is quite important, is that a processor is something that happens ahead of time. So we basically pre-process the data frame rather than doing it as we go. 

In [0]:
test = TabularList.from_df(df.iloc[800:1000].copy(), path=path, cat_names=cat_names, cont_names=cont_names)

**iloc[,] vs. loc[,]**

iloc[,] : int로 표현된 위치값으로 행/열/값을 추출하기 위한 방법. 마지막 값은 포함이 안됨. 즉 여기서 1000번째 행 data는 포함이 되지 않음. 

iloc는 row 번호 기준(위치 번호 기준)으로 데이터를 가져온다. loc는 index 번호 기준으로 데이터를 가져온다. 

iloc[1, 2, 9] : 1, 3, 5, 7의 index가 추출. loc[1, 2, 9] : 1, 3, 5, 7, 9의 index가 추출. (가장 객관적인 차이점인 것 같다.) 

loc는 string으로 된 column의 data를 뽑고 싶을 때, index가 정상적으로 잘 살아있을 때 쓰는 것 같음. index=6 없으면, 즉 index 하나라도 없으면 에러난다고 함. ㅎㅎ..까다롭..

In [0]:
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(800,1000)))
                           .label_from_df(cols=dep_var) # dep_var = 'salary'. 
                           .add_test(test) # where is test....
                           .databunch())

In [47]:
# cd /root/.fastai/data/adult_sample/models
ls

mini_train.pth


In [13]:
list(range(800, 1000)) 
# 800부터 999까지 int로 이루어진 list를 만들더라. index로 들어가는 듯  
len(list(range(800, 1000)))
# 길이는 200임. 숫자도 10에서 19까지 10개임.  

200

**the dependent variable is whether they're making over $50,000 salary. That's the thing we're trying to predict.**

In [19]:
# df = pd.read_csv(path/'texts.csv')
df.head()

# age workclass fnlwgt education education-num marital-status occupation relationship race sex capital-gain capital-loss hours-per-week native-countru "salary = dev_var = target"

# workclass	education	marital-status	occupation	relationship race : categorical values
# education-num_na(eduction_num에서 Nan을 구한듯함.?) age	fnlwgt	education-num	: continuous values
# target : derivative var(dev_var), 여기선 salary.  

# procs를 통해 int 값들이 normalize되었고, continuous value가 값을 가지지 않을 때 False로 만들어버림. 그에 반해 categorical value의 값이 없을 때는 ? 처리.  

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [15]:
data.show_batch(rows=10)

workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,target
Private,Some-college,Never-married,Transport-moving,Own-child,White,False,-0.9959,0.0209,-0.0312,<50k
Local-gov,Some-college,Married-spouse-absent,Craft-repair,Other-relative,White,False,0.6166,-0.2572,-0.0312,<50k
Private,Bachelors,Married-civ-spouse,Exec-managerial,Wife,Black,False,0.2502,0.6168,1.1422,>=50k
Private,Bachelors,Married-civ-spouse,Adm-clerical,Husband,White,False,-0.0430,1.0214,1.1422,>=50k
Private,12th,Never-married,Handlers-cleaners,Own-child,White,False,-1.5090,2.2119,-0.8135,<50k
Private,9th,Widowed,Exec-managerial,Not-in-family,Black,False,1.6428,0.0161,-1.9869,<50k
Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,False,-0.7027,-0.7103,-0.0312,<50k
Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Wife,White,False,0.9098,0.0880,1.1422,>=50k
Self-emp-not-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,False,0.3235,-1.4910,-0.4224,<50k
?,HS-grad,Widowed,?,Not-in-family,White,False,1.2030,-0.8160,-0.4224,<50k


In [0]:
learn = tabular_learner(data, layers=[200,100], metrics=accuracy)
# this is where we’re defining our architecture just like when we chose ResNet 34 or whatever for convolutional nets. 
# It refers to the sizes of the hidden fully connected layers between the input (after embedding) and before the classification layer. 
# The number of hidden layers decides the length of the list.

In [31]:
learn.model

TabularModel(
  (embeds): ModuleList(
    (0): Embedding(10, 6)
    (1): Embedding(17, 8)
    (2): Embedding(8, 5)
    (3): Embedding(16, 8)
    (4): Embedding(7, 5)
    (5): Embedding(6, 4)
    (6): Embedding(3, 3)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): Linear(in_features=42, out_features=200, bias=True)
    (1): ReLU(inplace=True)
    (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=200, out_features=100, bias=True)
    (4): ReLU(inplace=True)
    (5): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=100, out_features=2, bias=True)
  )
)

In [26]:
learn.fit(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.358505,0.356485,0.840000,00:04


## Inference

In [0]:
row = df.iloc[0]

In [29]:
row
# salary >= 50k 

age                                49
workclass                     Private
fnlwgt                         101320
education                  Assoc-acdm
education-num                      12
marital-status     Married-civ-spouse
occupation                        NaN
relationship                     Wife
race                            White
sex                            Female
capital-gain                        0
capital-loss                     1902
hours-per-week                     40
native-country          United-States
salary                          >=50k
Name: 0, dtype: object

In [28]:
learn.predict(row) 
# real value와 predicted value가 일치하는 것을 볼 수 있음. 

(Category >=50k, tensor(1), tensor([0.4359, 0.5641]))

In [0]:
test_row = df.iloc[500]

In [33]:
test_row

age                                36
workclass                     Private
fnlwgt                         223433
education                     Masters
education-num                     NaN
marital-status     Married-civ-spouse
occupation                      Sales
relationship                  Husband
race                            White
sex                              Male
capital-gain                     7688
capital-loss                        0
hours-per-week                     50
native-country          United-States
salary                          >=50k
Name: 500, dtype: object

In [35]:
learn.predict(test_row)

(Category >=50k, tensor(1), tensor([0.4037, 0.5963]))